In [1]:
%%time
import pandas as pd
#from sklearn import cross_validation
#from sklearn.feature_selection import SelectPercentile, f_classif
#from sklearn.linear_model import Lasso
#from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.naive_bayes import GaussianNB
from sklearn import tree
#from sklearn.svm import SVC
from sklearn import ensemble
#from sklearn.naive_bayes import BernoulliNB
#from datetime import datetime
#from blaze import Data
#import dask.dataframe as dd
#import dask.array as da

CPU times: user 1.13 s, sys: 519 ms, total: 1.64 s
Wall time: 4.06 s


In [2]:
%%time
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

CPU times: user 14 µs, sys: 3 µs, total: 17 µs
Wall time: 18.1 µs


In [3]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes)
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

CPU times: user 1min 56s, sys: 1min 34s, total: 3min 31s
Wall time: 3min 40s


In [4]:
%%time
all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(all_train['orig_destination_distance'].median()).astype(int)
all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
all_train['activity_month'] = all_train['date_time'].fillna(0).dt.month.astype(int)
all_train['checkin_month'] = all_train['srch_ci'].fillna(0).dt.month.astype(int)
all_train['checkout_month'] = all_train['srch_co'].fillna(0).dt.month.astype(int)

CPU times: user 32.8 s, sys: 5.22 s, total: 38 s
Wall time: 38.8 s


In [5]:
%%time
features = all_train[['activity_month', 'is_booking', 'user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance', 'checkin_month', 'checkout_month']]
labels = all_train['hotel_cluster']
features = features.reindex_axis(sorted(features.columns), axis=1)

CPU times: user 8.72 s, sys: 28.1 s, total: 36.8 s
Wall time: 48.7 s


In [6]:
features = features.values
del all_train

In [ ]:
%%time
#features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.25, random_state=42)
#selector = SelectKBest(k=5)
#.fit_transform(X, y)
#X_new.shape

#selector.fit(features, labels.values.ravel())
#feature_list = selector.transform(features)

In [8]:
%%time
clf = tree.DecisionTreeClassifier(min_samples_split=500)
#clf = GaussianNB()
#clf = ensemble.AdaBoostClassifier(n_estimators=5)
#clf = ensemble.AdaBoostClassifier(SVC(probability=True, kernel='linear'),n_estimators=10)
#clf = ensemble.GradientBoostingClassifier(SVC(probability=True, kernel='linear'),n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
#clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=3)
#clf = ensemble.RandomForestClassifier(n_estimators=5)
#clf - BernoulliNB()
clf = clf.fit(features, labels.values.ravel())

CPU times: user 6min 37s, sys: 6.15 s, total: 6min 43s
Wall time: 6min 45s


In [ ]:
#print accuracy_score(pred, labels_test.values.ravel())

In [9]:
%%time
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

CPU times: user 12 µs, sys: 12 µs, total: 24 µs
Wall time: 24.8 µs


In [10]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

CPU times: user 7.16 s, sys: 1.2 s, total: 8.35 s
Wall time: 8.5 s


In [11]:
%%time
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(all_test['orig_destination_distance'].median()).astype(int)
all_test['date_time'] = pd.to_datetime(all_test['date_time'], errors='coerce')
all_test['srch_ci'] = pd.to_datetime(all_test['srch_ci'], errors='coerce')
all_test['srch_co'] = pd.to_datetime(all_test['srch_co'], errors='coerce')
all_test['activity_month'] = all_test['date_time'].fillna(0).dt.month.astype(int)
all_test['checkin_month'] = all_test['srch_ci'].fillna(0).dt.month.astype(int)
all_test['checkout_month'] = all_test['srch_co'].fillna(0).dt.month.astype(int)
all_test['is_booking'] = 1

CPU times: user 2.38 s, sys: 200 ms, total: 2.58 s
Wall time: 2.61 s


In [12]:
testing_features = all_test[['activity_month', 'is_booking', 'user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance', 'checkin_month', 'checkout_month']]
testing_features = testing_features.reindex_axis(sorted(testing_features.columns), axis=1)

In [13]:
%%time
#feature_test_file = selector.transform(all_test)
#pred = clf.predict(feature_test_list)
probs = pd.DataFrame(clf.predict_proba(testing_features.values))

CPU times: user 3.01 s, sys: 1.57 s, total: 4.58 s
Wall time: 4.9 s


In [ ]:
%%time
probs_series = pd.Series([(i, r.sort_values(ascending=False)[:5].index.values) for i,r in probs.iterrows()])
probs_series = probs_series.values

In [ ]:
%%time
indices = [a for a,b in probs_series]
values = [b for a,b in probs_series]

In [ ]:
%%time
submission = pd.DataFrame()
submission['id'] = indices
submission['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values]

In [ ]:
%%time
submission.to_csv('submission.csv', index=False)